# LangGraph MCP Agent - Quick Start

This notebook demonstrates how to use the modular LangGraph MCP agent package.

**Note:** This notebook uses the refactored `langgraph_agent` package with proper separation of concerns.

## 1. Setup and Configuration

**Important:** After running the install cell below, restart the kernel before proceeding.


In [1]:
# Install the local package in development mode
import sys
import subprocess

# Install from parent directory
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-e", ".."])
print("✓ Package installed. Please restart the kernel to use the package.")


✓ Package installed. Please restart the kernel to use the package.


In [2]:
# Import the agent package
from langgraph_agent import get_config, initialize_agent
from langgraph_agent.utils import get_workspace_client, setup_mlflow_tracking

# Load configuration
config = get_config()

# Show current configuration
print(f"Model Endpoint: {config.model.endpoint_name}")
print(f"Databricks Profile: {config.databricks.profile}")
print(f"MLflow Experiment: {config.mlflow.experiment_name}")
print(f"UC Model: {config.uc.full_model_name}")


Model Endpoint: databricks-claude-3-7-sonnet
Databricks Profile: development
MLflow Experiment: /Shared/langgraph-mcp-agent
UC Model: rag.development.langgraph_mcp_agent


## 2. Test Authentication

In [3]:
from langgraph_agent.utils import verify_authentication

# Verify authentication
auth_info = verify_authentication(config.databricks.profile)
print(f"✓ Authenticated to: {auth_info['host']}")
print(f"✓ Profile: {auth_info['profile']}")
print(f"✓ Auth type: {auth_info['auth_type']}")

✓ Authenticated to: https://adb-3253299566947192.12.azuredatabricks.net
✓ Profile: development
✓ Auth type: metadata-service


## 3. Setup MLflow Tracking

In [5]:
# Setup MLflow tracking
mlflow_config = setup_mlflow_tracking(
    profile=config.databricks.profile,
    experiment_name=config.mlflow.experiment_name,
    enable_autolog=True
)

print(f"✓ Tracking URI: {mlflow_config.get('tracking_uri')}")
print(f"✓ Experiment: {mlflow_config.get('experiment_name')}")

✓ Tracking URI: databricks://development
✓ Experiment: None


## 4. Initialize the Agent

In [6]:
# Get workspace client
ws = get_workspace_client(config.databricks.profile)
ws_host = ws.config.host

# Build MCP server URLs
managed_urls = config.mcp.managed_urls or [
    f"{ws_host}/api/2.0/mcp/functions/system/ai"
]

print(f"MCP Servers: {managed_urls}")

# Initialize agent
agent = initialize_agent(
    workspace_client=ws,
    llm_endpoint_name=config.model.endpoint_name,
    system_prompt=config.model.system_prompt,
    managed_mcp_urls=managed_urls,
    custom_mcp_urls=config.mcp.custom_urls,
)

print("✓ Agent initialized successfully!")

MCP Servers: ['https://adb-3253299566947192.12.azuredatabricks.net/api/2.0/mcp/functions/system/ai']
✓ Agent initialized successfully!
✓ Agent initialized successfully!


## 5. Test the Agent

In [7]:
# Test with a simple query
response = agent.predict({
    "input": [
        {"role": "user", "content": "What is 7*6 in Python?"}
    ]
})

print(response)

tool_choice=None truncation=None id=None created_at=None error=None incomplete_details=None instructions=None metadata=None model=None object='response' output=[OutputItem(type='message', id='lc_run--829868ec-2eab-4246-834f-70cd5fe28bf0', content=[{'text': 'I can calculate 7*6 using Python. Let me run that for you:', 'type': 'output_text'}], role='assistant'), OutputItem(type='function_call', id='lc_run--829868ec-2eab-4246-834f-70cd5fe28bf0', call_id='toolu_bdrk_012Wvos1WKfJ54FBdL6nFsjK', name='system__ai__python_exec', arguments='{"code": "print(7 * 6)"}'), OutputItem(type='function_call_output', call_id='toolu_bdrk_012Wvos1WKfJ54FBdL6nFsjK', output='{"is_truncated":false,"columns":["output"],"rows":[["42\\n"]]}'), OutputItem(type='message', id='lc_run--534e59fd-9adc-4c66-9142-9462624cd9a5', content=[{'text': 'The result of 7*6 in Python is 42.', 'type': 'output_text'}], role='assistant')] parallel_tool_calls=None temperature=None tools=None top_p=None max_output_tokens=None previous_

2025/11/08 23:49:34 WARNING mlflow.tracing.export.mlflow_v3: Failed to send trace to MLflow backend: INVALID_PARAMETER_VALUE: `trace_info.mlflow_experiment.experiment_id` is missing.


In [ ]:
# Test with streaming
print("Streaming response:")
print("-" * 60)

for chunk in agent.predict_stream({
    "input": [
        {"role": "user", "content": "Calculate the 15th Fibonacci number using Python"}
    ]
}):
    print(chunk)

## 6. Evaluate the Agent

In [ ]:
from langgraph_agent.evaluate import run_evaluation_pipeline

# Run evaluation
eval_results = run_evaluation_pipeline(
    agent=agent,
    dataset_path="data/eval_dataset.json",
    experiment_name=config.mlflow.experiment_name,
)

print(f"\nEvaluation Results:")
print(f"Dataset size: {eval_results['dataset_size']}")
print(f"Metrics: {eval_results['metrics']}")

## 7. Deploy to Unity Catalog

**Note:** This step requires the agent code to be in a Python module file.

In [ ]:
from langgraph_agent.deploy import full_deployment_pipeline

# Run full deployment pipeline
deployment_result = full_deployment_pipeline(
    config=config,
    validate=True,
)

print("\n✓ Deployment Complete!")
print(f"Model: {deployment_result['registered_model']['name']}")
print(f"Version: {deployment_result['registered_model']['version']}")
print(f"Deployment: {deployment_result['deployment']}")

## Next Steps

After deployment:
1. Test the agent in AI Playground
2. Share with SMEs for feedback
3. Monitor performance and traces
4. Iterate based on feedback

You can also use the CLI for these operations:
```bash
# Evaluate
langgraph-agent evaluate --dataset data/eval_dataset.json

# Deploy
langgraph-agent deploy

# Check configuration
langgraph-agent config-show
```